# Lección 1: NumPy - Generación de Datos

In [1]:
import numpy as np

np.random.seed(42)
clientes = np.arange(100, 200)
transacciones = np.random.randint(1, 20, size=100)
montos = np.random.normal(loc=50000, scale=15000, size=100).round(2)

print("Monto promedio:", np.mean(montos))
print("Total transacciones:", np.sum(transacciones))

np.save("clientes.npy", clientes)
np.save("transacciones.npy", transacciones)
np.save("montos.npy", montos)

Monto promedio: 50801.509000000005
Total transacciones: 964


# Lección 2: Pandas - Exploración de Datos

In [2]:
import pandas as pd

clientes = np.load("clientes.npy")
transacciones = np.load("transacciones.npy")
montos = np.load("montos.npy")

df = pd.DataFrame({
    "cliente_id": clientes,
    "transacciones": transacciones,
    "monto_total": montos
})

print(df.head())
print(df.describe())
print(df[df["monto_total"] > 70000])

df.to_csv("datos_clientes.csv", index=False)

   cliente_id  transacciones  monto_total
0         100              7     56868.28
1         101             15     24735.69
2         102             11     54902.68
3         103              8     48783.22
4         104              7     57016.92
       cliente_id  transacciones   monto_total
count  100.000000     100.000000    100.000000
mean   149.500000       9.640000  50801.509000
std     29.011492       5.296311  14934.313948
min    100.000000       1.000000  20901.330000
25%    124.750000       5.000000  39036.797500
50%    149.500000       9.000000  49870.225000
75%    174.250000      14.000000  61944.390000
max    199.000000      19.000000  94154.950000
    cliente_id  transacciones  monto_total
12         112             12     89502.92
17         117             12     84308.39
18         118             17     74343.81
37         137             15     94154.95
55         155              2     79618.57
56         156             10     79475.52
63         163          

# Lección 3: Carga de Datos desde Archivos

In [5]:
df_csv = pd.read_csv("clientes_ecommerce.csv")
df_excel = pd.read_excel("clientes_ecommerce.xlsx")

# Usamos la tabla web de ejemplo
df_web = pd.read_html("https://es.wikipedia.org/wiki/Anexo:Comunas_de_Chile")[0]

df_merged = pd.concat([df_csv, df_excel], ignore_index=True)
df_merged.to_csv("dataset_unificado.csv", index=False)
df_merged.head()

,ID,Nombre,Edad,Ciudad,Total_Compras,Monto_Total
0,1,Ana,25.0,Buenos Aires,5,2500
1,2,Luis,34.0,Córdoba,2,1200
2,3,Carlos,45.0,Rosario,0,0
3,4,Marta,29.0,Mendoza,3,1800
4,5,Jorge,31.0,La Plata,8,3200


# Lección 4: Manejo de Valores Nulos y Outliers

In [8]:
from scipy import stats

print(df_merged.isnull().sum())
#df_merged["Monto_Total"].fillna(df_merged["Monto_Total"].mean(), inplace=True)
df_merged.fillna({"Monto_Total": df_merged["Monto_Total"].mean()}, inplace=True)

z_scores = np.abs(stats.zscore(df_merged[["Monto_Total"]]))
df_limpio = df_merged[(z_scores < 3).all(axis=1)]

df_limpio.to_csv("dataset_limpio.csv", index=False)
df_limpio.describe()

ID               0
Nombre           0
Edad             2
Ciudad           0
Total_Compras    0
Monto_Total      0
dtype: int64


,ID,Edad,Total_Compras,Monto_Total
count,20.000000,18.00000,20.000000,20.000000
mean,5.500000,33.00000,4.600000,2155.000000
std,2.946898,6.36165,3.118704,1285.639304
min,1.000000,25.00000,0.000000,0.000000
25%,3.000000,29.00000,2.000000,1200.000000
50%,5.500000,31.00000,4.500000,2300.000000
75%,8.000000,34.00000,7.000000,3200.000000
max,10.000000,45.00000,10.000000,4100.000000


# Lección 5: Data Wrangling

In [9]:
df_limpio.drop_duplicates(inplace=True)
df_limpio["ID"] = df_limpio["ID"].astype(str)
df_limpio["Monto_Promedio"] = df_limpio["Monto_Total"] / df_limpio["Total_Compras"]
df_limpio["Monto_Normalizado"] = (df_limpio["Monto_Total"] - df_limpio["Monto_Total"].min()) /                                   (df_limpio["Monto_Total"].max() - df_limpio["Monto_Total"].min())

df_limpio.to_csv("dataset_transformado.csv", index=False)
df_limpio.head()

,ID,Nombre,Edad,Ciudad,Total_Compras,Monto_Total,Monto_Promedio,Monto_Normalizado
0,1,Ana,25.0,Buenos Aires,5,2500,500.0,0.609756
1,2,Luis,34.0,Córdoba,2,1200,600.0,0.292683
2,3,Carlos,45.0,Rosario,0,0,NaN,0.000000
3,4,Marta,29.0,Mendoza,3,1800,600.0,0.439024
4,5,Jorge,31.0,La Plata,8,3200,400.0,0.780488


# Lección 6: Agrupamiento y Pivoteo

In [12]:
df_grouped = df_limpio.groupby("Total_Compras").agg({
    "Monto_Total": ["mean", "sum"],
    "ID": "count"
}).reset_index()

pivot = df_limpio.pivot_table(values="Monto_Total", index="Total_Compras", aggfunc="mean")

df_grouped.to_csv("agrupamiento.csv", index=False)
pivot.to_excel("pivot_resultado.xlsx")
df_limpio.to_csv("dataset_final.csv", index=False)
df_limpio.to_excel("dataset_final.xlsx", index=False)
